In [1]:
# Cell 1 - Setup and Data Loading
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

print("⚙️ XO Project - Feature Engineering Phase")
print("="*60)
print("Objective: Create physics-based features for habitability classification")
print("="*60)

# Load cleaned data from previous phase
df_clean = pd.read_csv('../data/processed/cleaned_data.csv')
print(f"Loaded cleaned dataset: {len(df_clean):,} planets, {len(df_clean.columns)} features")

# Display basic info about the dataset
print(f"\nDataset overview:")
print(f"Date range: {df_clean['disc_year'].min()} - {df_clean['disc_year'].max()}")
print(f"Available features: {df_clean.columns.tolist()}")

# Check completeness of key features for feature engineering
key_features = ['pl_rade', 'pl_orbsmax', 'st_teff', 'st_mass']
print(f"\nKey features completeness:")
for feature in key_features:
    completeness = (df_clean[feature].count() / len(df_clean)) * 100
    print(f"{feature:12} | {completeness:5.1f}% complete")

⚙️ XO Project - Feature Engineering Phase
Objective: Create physics-based features for habitability classification
Loaded cleaned dataset: 1,729 planets, 15 features

Dataset overview:
Date range: 2004 - 2025
Available features: ['pl_name', 'hostname', 'pl_rade', 'pl_bmasse', 'pl_orbsmax', 'st_teff', 'st_mass', 'pl_eqt', 'discoverymethod', 'disc_year', 'pl_orbper', 'pl_orbeccen', 'st_rad', 'st_logg', 'pl_insol']

Key features completeness:
pl_rade      | 100.0% complete
pl_orbsmax   | 100.0% complete
st_teff      | 100.0% complete
st_mass      | 100.0% complete


In [2]:
# Cell 2 - Habitable Zone Calculations
print("\nHabitable Zone Feature Engineering:")
print("="*50)

# Copy dataframe for feature engineering
df_features = df_clean.copy()

def calculate_habitable_zone_features(row):
    """
    Calculate habitable zone related features for a planet
    
    Physics:
    - Conservative HZ: 0.95 to 1.37 AU (scaled by stellar luminosity)
    - Stellar luminosity: L ∝ M^3.5 (main sequence approximation)
    - HZ boundaries scale as sqrt(L_star/L_sun)
    """
    
    # Extract required values
    orbital_distance = row['pl_orbsmax']
    stellar_mass = row['st_mass']
    
    # Handle missing values
    if pd.isna(orbital_distance) or pd.isna(stellar_mass):
        return pd.Series({
            'stellar_luminosity': np.nan,
            'hz_inner': np.nan,
            'hz_outer': np.nan,
            'hz_center': np.nan,
            'hz_width': np.nan,
            'hz_position': np.nan,
            'in_habitable_zone': False,
            'hz_distance_factor': np.nan
        })
    
    # Calculate stellar luminosity (main sequence approximation)
    stellar_luminosity = stellar_mass ** 3.5
    
    # Calculate habitable zone boundaries (AU)
    hz_inner = 0.95 * np.sqrt(stellar_luminosity)
    hz_outer = 1.37 * np.sqrt(stellar_luminosity)
    hz_center = (hz_inner + hz_outer) / 2
    hz_width = hz_outer - hz_inner
    
    # Calculate planet's position relative to HZ
    hz_position = orbital_distance / hz_center  # 1.0 = center of HZ
    
    # Check if planet is in habitable zone
    in_hz = (orbital_distance >= hz_inner) and (orbital_distance <= hz_outer)
    
    # Distance factor: how far from optimal position (logarithmic scale)
    hz_distance_factor = np.log10(hz_position) if hz_position > 0 else np.nan
    
    return pd.Series({
        'stellar_luminosity': stellar_luminosity,
        'hz_inner': hz_inner,
        'hz_outer': hz_outer,
        'hz_center': hz_center,
        'hz_width': hz_width,
        'hz_position': hz_position,
        'in_habitable_zone': in_hz,
        'hz_distance_factor': hz_distance_factor
    })

# Apply habitable zone calculations
print("Calculating habitable zone features...")
hz_features = df_features.apply(calculate_habitable_zone_features, axis=1)
df_features = pd.concat([df_features, hz_features], axis=1)

# Summary of habitable zone analysis
valid_hz = df_features['in_habitable_zone'].notna()
hz_planets = df_features['in_habitable_zone'].sum()
print(f"Planets with HZ analysis: {valid_hz.sum():,}")
print(f"Planets in habitable zone: {hz_planets:,} ({hz_planets/valid_hz.sum()*100:.1f}%)")

# Show examples of planets in HZ
if hz_planets > 0:
    hz_examples = df_features[df_features['in_habitable_zone'] == True].head()
    print(f"\nExample planets in habitable zone:")
    for _, planet in hz_examples.iterrows():
        print(f"  {planet['pl_name']:20} | {planet['pl_orbsmax']:.3f} AU | HZ: {planet['hz_inner']:.3f}-{planet['hz_outer']:.3f} AU")


Habitable Zone Feature Engineering:
Calculating habitable zone features...
Planets with HZ analysis: 1,729
Planets in habitable zone: 11 (0.6%)

Example planets in habitable zone:
  Kepler-315 c         | 0.760 AU | HZ: 0.687-0.991 AU
  Kepler-1554 b        | 0.595 AU | HZ: 0.529-0.764 AU
  Kepler-1868 b        | 0.623 AU | HZ: 0.541-0.780 AU
  Kepler-1600 b        | 0.981 AU | HZ: 0.706-1.018 AU
  Kepler-1636 b        | 1.122 AU | HZ: 1.016-1.465 AU


In [3]:
# Cell 3 - Earth Similarity Index (ESI)
print("\nEarth Similarity Index Engineering:")
print("="*50)

def calculate_earth_similarity_index(row):
    """
    Calculate Earth Similarity Index components
    
    ESI formula: ESI = (ESI_radius × ESI_mass × ESI_temperature)^(1/3)
    Where each component: ESI_x = 1 - |x_planet - x_earth| / (x_planet + x_earth)
    
    Earth reference values:
    - Radius: 1.0 Earth radii
    - Mass: 1.0 Earth masses  
    - Temperature: 288 K (Earth's mean surface temperature)
    """
    
    # Earth reference values
    earth_radius = 1.0
    earth_mass = 1.0
    earth_temp = 288.0  # Kelvin
    
    # Extract planetary values
    planet_radius = row['pl_rade']
    planet_mass = row['pl_bmasse'] 
    planet_temp = row['pl_eqt']
    
    # Calculate ESI components
    esi_radius = np.nan
    esi_mass = np.nan  
    esi_temperature = np.nan
    
    if not pd.isna(planet_radius):
        esi_radius = 1 - abs(planet_radius - earth_radius) / (planet_radius + earth_radius)
        
    if not pd.isna(planet_mass):
        esi_mass = 1 - abs(planet_mass - earth_mass) / (planet_mass + earth_mass)
        
    if not pd.isna(planet_temp):
        esi_temperature = 1 - abs(planet_temp - earth_temp) / (planet_temp + earth_temp)
    
    # Calculate composite ESI (only if all components available)
    esi_composite = np.nan
    if all(not pd.isna(x) for x in [esi_radius, esi_mass, esi_temperature]):
        esi_composite = (esi_radius * esi_mass * esi_temperature) ** (1/3)
    
    # Surface ESI (radius + temperature only - more commonly available)
    esi_surface = np.nan
    if all(not pd.isna(x) for x in [esi_radius, esi_temperature]):
        esi_surface = (esi_radius * esi_temperature) ** (1/2)
    
    return pd.Series({
        'esi_radius': esi_radius,
        'esi_mass': esi_mass,
        'esi_temperature': esi_temperature,
        'esi_composite': esi_composite,
        'esi_surface': esi_surface
    })

# Apply ESI calculations
print("Calculating Earth Similarity Index...")
esi_features = df_features.apply(calculate_earth_similarity_index, axis=1)
df_features = pd.concat([df_features, esi_features], axis=1)

# ESI analysis summary
esi_available = df_features['esi_surface'].notna().sum()
high_esi = (df_features['esi_surface'] > 0.8).sum()
print(f"Planets with surface ESI: {esi_available:,}")
print(f"High ESI planets (>0.8): {high_esi:,}")

if high_esi > 0:
    print("\nTop Earth-like planets by surface ESI:")
    top_esi = df_features.nlargest(5, 'esi_surface')[['pl_name', 'pl_rade', 'pl_eqt', 'esi_surface']]
    for _, planet in top_esi.iterrows():
        print(f"  {planet['pl_name']:20} | ESI: {planet['esi_surface']:.3f} | R: {planet['pl_rade']:.2f} | T: {planet['pl_eqt']:.0f}K")




Earth Similarity Index Engineering:
Calculating Earth Similarity Index...
Planets with surface ESI: 368
High ESI planets (>0.8): 11

Top Earth-like planets by surface ESI:
  LP 791-18 d          | ESI: 0.911 | R: 1.03 | T: 396K
  LP 890-9 c           | ESI: 0.906 | R: 1.37 | T: 272K
  K2-239 c             | ESI: 0.898 | R: 1.00 | T: 427K
  K2-239 d             | ESI: 0.894 | R: 1.10 | T: 399K
  K2-3 d               | ESI: 0.889 | R: 1.46 | T: 305K


In [4]:
# Cell 4 - Atmospheric Retention Features
print("\nAtmospheric Retention Features:")
print("="*50)

def calculate_atmospheric_features(row):
    """
    Calculate features related to atmospheric retention
    
    Physics:
    - Escape velocity: v_esc = sqrt(2GM/R)  
    - Higher mass and smaller radius = better atmosphere retention
    - Stellar flux affects atmospheric loss
    """
    
    # Constants
    G = 6.67e-11  # Gravitational constant (m^3 kg^-1 s^-2)
    earth_mass_kg = 5.97e24  # kg
    earth_radius_m = 6.37e6  # m
    
    # Extract values
    planet_mass = row['pl_bmasse']  # Earth masses
    planet_radius = row['pl_rade']  # Earth radii
    stellar_flux = row['pl_insol'] if 'pl_insol' in row else np.nan  # Earth flux units
    
    # Calculate escape velocity (in km/s, relative to Earth's 11.2 km/s)
    escape_velocity_ratio = np.nan
    if not pd.isna(planet_mass) and not pd.isna(planet_radius) and planet_radius > 0:
        # v_esc ∝ sqrt(M/R)
        escape_velocity_ratio = np.sqrt(planet_mass / planet_radius)
    
    # Atmospheric retention capability (qualitative)
    retention_capability = np.nan
    if not pd.isna(escape_velocity_ratio):
        if escape_velocity_ratio >= 1.0:  # Earth-like or better
            retention_capability = 1.0
        elif escape_velocity_ratio >= 0.5:  # Moderate retention (like Mars)
            retention_capability = 0.5
        else:  # Poor retention (like Moon)
            retention_capability = 0.1
    
    # Stellar flux factor (Earth = 1.0)
    flux_factor = stellar_flux if not pd.isna(stellar_flux) else np.nan
    
    return pd.Series({
        'escape_velocity_ratio': escape_velocity_ratio,
        'atmospheric_retention': retention_capability,
        'stellar_flux': flux_factor
    })

# Apply atmospheric calculations
print("Calculating atmospheric retention features...")
atm_features = df_features.apply(calculate_atmospheric_features, axis=1)
df_features = pd.concat([df_features, atm_features], axis=1)

# Atmospheric analysis summary
good_retention = (df_features['atmospheric_retention'] >= 1.0).sum()
moderate_retention = ((df_features['atmospheric_retention'] >= 0.5) & 
                     (df_features['atmospheric_retention'] < 1.0)).sum()
print(f"Planets with good atmospheric retention: {good_retention:,}")
print(f"Planets with moderate atmospheric retention: {moderate_retention:,}")


Atmospheric Retention Features:
Calculating atmospheric retention features...
Planets with good atmospheric retention: 296
Planets with moderate atmospheric retention: 6


In [5]:
# Cell 5 - Stellar Habitability Features  
print("\nStellar Habitability Features:")
print("="*50)

def calculate_stellar_features(row):
    """
    Calculate features related to stellar habitability
    
    Considerations:
    - Stellar temperature affects habitable zone
    - Stellar mass affects lifetime and stability
    - Stellar radius affects luminosity
    """
    
    # Extract stellar properties
    stellar_temp = row['st_teff']
    stellar_mass = row['st_mass']
    stellar_radius = row['st_rad'] if 'st_rad' in row else np.nan
    
    # Stellar classification based on temperature
    stellar_class = 'Unknown'
    temp_habitability = 0.0
    
    if not pd.isna(stellar_temp):
        if 2500 <= stellar_temp < 3700:  # M dwarf
            stellar_class = 'M'
            temp_habitability = 0.6  # Concerns about flares, tidal locking
        elif 3700 <= stellar_temp < 5200:  # K dwarf  
            stellar_class = 'K'
            temp_habitability = 0.9  # Very good for habitability
        elif 5200 <= stellar_temp < 6000:  # G dwarf (Sun-like)
            stellar_class = 'G'
            temp_habitability = 1.0  # Optimal
        elif 6000 <= stellar_temp < 7500:  # F dwarf
            stellar_class = 'F'
            temp_habitability = 0.7  # Shorter lifetime, more active
        else:
            temp_habitability = 0.2  # Too hot or too cool
    
    # Stellar mass habitability (optimal around solar mass)
    mass_habitability = 0.0
    if not pd.isna(stellar_mass):
        if 0.5 <= stellar_mass <= 1.2:  # Good range
            mass_habitability = 1.0 - abs(stellar_mass - 1.0) * 0.5
        else:
            mass_habitability = 0.3
    
    # Stellar lifetime estimate (main sequence, in Gyr)
    stellar_lifetime = np.nan
    if not pd.isna(stellar_mass) and stellar_mass > 0:
        stellar_lifetime = 10 * (stellar_mass ** -2.5)  # Rough approximation
    
    return pd.Series({
        'stellar_class': stellar_class,
        'stellar_temp_habitability': temp_habitability,
        'stellar_mass_habitability': mass_habitability,
        'stellar_lifetime_gyr': stellar_lifetime
    })

# Apply stellar calculations
print("Calculating stellar habitability features...")
stellar_features = df_features.apply(calculate_stellar_features, axis=1)
df_features = pd.concat([df_features, stellar_features], axis=1)

# Stellar analysis summary
stellar_classes = df_features['stellar_class'].value_counts()
print("Stellar class distribution:")
for star_class, count in stellar_classes.items():
    print(f"  {star_class} class: {count:,} stars")

good_stars = (df_features['stellar_temp_habitability'] >= 0.8).sum()
print(f"Stars with good habitability potential: {good_stars:,}")


Stellar Habitability Features:
Calculating stellar habitability features...
Stellar class distribution:
  G class: 969 stars
  K class: 382 stars
  F class: 316 stars
  M class: 58 stars
  Unknown class: 4 stars
Stars with good habitability potential: 1,351


In [6]:
# Cell 6 - Composite Habitability Features
print("\nComposite Habitability Score:")
print("="*50)

def calculate_habitability_score(row):
    """
    Calculate overall habitability score combining multiple factors
    
    Factors considered:
    1. Habitable zone position (0-1)
    2. Earth similarity (0-1) 
    3. Atmospheric retention (0-1)
    4. Stellar habitability (0-1)
    """
    
    # Individual factor scores
    hz_score = 0.0
    if row['in_habitable_zone']:
        hz_score = 1.0
    elif not pd.isna(row['hz_position']):
        # Partial credit for being close to HZ
        if 0.5 <= row['hz_position'] <= 2.0:
            hz_score = 1.0 - abs(np.log10(row['hz_position'])) / np.log10(2)
    
    esi_score = row['esi_surface'] if not pd.isna(row['esi_surface']) else 0.0
    
    atm_score = row['atmospheric_retention'] if not pd.isna(row['atmospheric_retention']) else 0.5
    
    stellar_score = row['stellar_temp_habitability'] if not pd.isna(row['stellar_temp_habitability']) else 0.5
    
    # Weighted composite score
    weights = {
        'hz': 0.4,      # Habitable zone most important
        'esi': 0.3,     # Earth similarity important
        'atm': 0.2,     # Atmospheric retention
        'stellar': 0.1  # Stellar properties
    }
    
    composite_score = (weights['hz'] * hz_score + 
                      weights['esi'] * esi_score + 
                      weights['atm'] * atm_score + 
                      weights['stellar'] * stellar_score)
    
    return pd.Series({
        'hz_score': hz_score,
        'esi_score': esi_score,
        'atm_score': atm_score,
        'stellar_score': stellar_score,
        'habitability_score': composite_score
    })

# Apply composite score calculation
print("Calculating composite habitability scores...")
hab_scores = df_features.apply(calculate_habitability_score, axis=1)
df_features = pd.concat([df_features, hab_scores], axis=1)

# Habitability analysis
score_stats = df_features['habitability_score'].describe()
print(f"Habitability score statistics:")
print(f"  Mean: {score_stats['mean']:.3f}")
print(f"  Std:  {score_stats['std']:.3f}")
print(f"  Max:  {score_stats['max']:.3f}")

# Top habitable candidates
print(f"\nTop 10 most habitable planets:")
top_habitable = df_features.nlargest(10, 'habitability_score')
for _, planet in top_habitable.iterrows():
    print(f"  {planet['pl_name']:20} | Score: {planet['habitability_score']:.3f} | HZ: {planet['in_habitable_zone']}")




Composite Habitability Score:
Calculating composite habitability scores...
Habitability score statistics:
  Mean: 0.246
  Std:  0.099
  Max:  0.939

Top 10 most habitable planets:
  Kepler-22 b          | Score: 0.939 | HZ: True
  LP 890-9 c           | Score: 0.719 | HZ: False
  Kepler-34 b          | Score: 0.700 | HZ: True
  Kepler-1661 b        | Score: 0.699 | HZ: False
  LHS 1140 b           | Score: 0.640 | HZ: False
  Kepler-315 c         | Score: 0.600 | HZ: True
  Kepler-1636 b        | Score: 0.600 | HZ: True
  Kepler-150 f         | Score: 0.600 | HZ: True
  Kepler-1554 b        | Score: 0.590 | HZ: True
  Kepler-1868 b        | Score: 0.590 | HZ: True


In [7]:
# Cell 7 - Feature Summary and Export
print("\nFeature Engineering Summary:")
print("="*50)

# Count engineered features
original_features = len(df_clean.columns)
engineered_features = len(df_features.columns) - original_features
print(f"Original features: {original_features}")
print(f"Engineered features: {engineered_features}")
print(f"Total features: {len(df_features.columns)}")

# List all engineered features
new_features = [col for col in df_features.columns if col not in df_clean.columns]
print(f"\nEngineered features:")
for i, feature in enumerate(new_features, 1):
    print(f"  {i:2d}. {feature}")

# Data completeness after feature engineering
print(f"\nFeature completeness:")
key_engineered = ['in_habitable_zone', 'esi_surface', 'habitability_score']
for feature in key_engineered:
    completeness = (df_features[feature].notna().sum() / len(df_features)) * 100
    print(f"{feature:20} | {completeness:5.1f}% complete")

# Save engineered features
output_path = '../data/processed/engineered_features.csv'
df_features.to_csv(output_path, index=False)
print(f"\nSaved engineered dataset to: {output_path}")

# Create feature documentation
doc_path = '../docs/feature_documentation.md'
with open(doc_path, 'w') as f:
    f.write("# XO Project - Feature Documentation\n\n")
    f.write("## Physics-Based Engineered Features\n\n")
    f.write("### Habitable Zone Features\n")
    f.write("- `stellar_luminosity`: Stellar luminosity relative to Sun\n")
    f.write("- `hz_inner/hz_outer`: Habitable zone boundaries (AU)\n") 
    f.write("- `hz_position`: Planet distance relative to HZ center\n")
    f.write("- `in_habitable_zone`: Boolean flag for HZ membership\n\n")
    f.write("### Earth Similarity Index\n")
    f.write("- `esi_radius/mass/temperature`: Individual ESI components\n")
    f.write("- `esi_surface`: Combined radius + temperature ESI\n\n")
    f.write("### Atmospheric Features\n") 
    f.write("- `escape_velocity_ratio`: Relative to Earth's escape velocity\n")
    f.write("- `atmospheric_retention`: Qualitative retention capability\n\n")
    f.write("### Habitability Score\n")
    f.write("- `habitability_score`: Weighted composite score (0-1)\n")

print(f"Created feature documentation: {doc_path}")
print(f"\nFeature engineering complete!")
print(f"Next phase: 04_habitability_labeling.ipynb")


Feature Engineering Summary:
Original features: 15
Engineered features: 25
Total features: 40

Engineered features:
   1. stellar_luminosity
   2. hz_inner
   3. hz_outer
   4. hz_center
   5. hz_width
   6. hz_position
   7. in_habitable_zone
   8. hz_distance_factor
   9. esi_radius
  10. esi_mass
  11. esi_temperature
  12. esi_composite
  13. esi_surface
  14. escape_velocity_ratio
  15. atmospheric_retention
  16. stellar_flux
  17. stellar_class
  18. stellar_temp_habitability
  19. stellar_mass_habitability
  20. stellar_lifetime_gyr
  21. hz_score
  22. esi_score
  23. atm_score
  24. stellar_score
  25. habitability_score

Feature completeness:
in_habitable_zone    | 100.0% complete
esi_surface          |  21.3% complete
habitability_score   | 100.0% complete

Saved engineered dataset to: ../data/processed/engineered_features.csv
Created feature documentation: ../docs/feature_documentation.md

Feature engineering complete!
Next phase: 04_habitability_labeling.ipynb
